<a href="https://colab.research.google.com/github/asifmanowar9/ML-Project/blob/main/resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load the Dataset

First, let's load your dataset using `tf.keras.utils.image_dataset_from_directory`. This function is very convenient for datasets organized into subdirectories per class.


In [6]:
import tensorflow as tf

# Define the base path to your dataset
dataset_base_path = '/content/drive/MyDrive/Datasets/ML Datasets/dataset'

# Define image dimensions and batch size
IMG_HEIGHT = 224 # Common size for many pre-trained models
IMG_WIDTH = 224  # Common size for many pre-trained models
BATCH_SIZE = 32

# Load the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_base_path + '/train',
    labels='inferred',
    label_mode='int', # Return integer encoded labels (0, 1, 2, ...)
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Load the validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_base_path + '/val',
    labels='inferred',
    label_mode='int',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=False # Typically no need to shuffle validation data
)

# Load the test dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_base_path + '/test',
    labels='inferred',
    label_mode='int',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=False # Typically no need to shuffle test data
)

print("Datasets loaded successfully!")

Found 10093 files belonging to 4 classes.
Found 2161 files belonging to 4 classes.
Found 2167 files belonging to 4 classes.
Datasets loaded successfully!


### Inspect the Dataset

Let's check the class names and the shape of a batch of images and labels.


In [13]:
class_names = train_ds.class_names
print(f"Class names: {class_names}")
print(f"Number of classes: {len(class_names)}")

for image_batch, labels_batch in train_ds.take(1):
    print(f"Image batch shape: {image_batch.shape}") # (batch_size, img_height, img_width, channels)
    print(f"Labels batch shape: {labels_batch.shape}") # (batch_size,)
    print(f"Sample labels: {labels_batch.numpy()}")

# You can also manually verify the number of samples in each dataset (optional)
# num_train_samples = tf.data.experimental.cardinality(train_ds).numpy() * BATCH_SIZE
# num_val_samples = tf.data.experimental.cardinality(val_ds).numpy() * BATCH_SIZE
# num_test_samples = tf.data.experimental.cardinality(test_ds).numpy() * BATCH_SIZE
# print(f"Number of training samples (approx): {num_train_samples}")
# print(f"Number of validation samples (approx): {num_val_samples}")
# print(f"Number of test samples (approx): {num_test_samples}")


Class names: ['100', '1000', '50', '500']
Number of classes: 4
Image batch shape: (32, 224, 224, 3)
Labels batch shape: (32,)
Sample labels: [3 1 1 1 2 1 2 2 3 1 0 2 3 1 1 3 3 3 0 3 3 1 3 0 0 1 3 1 0 0 1 1]


### Model Choice for Beginners (Image Classification)

For image classification, especially as a beginner, I highly recommend starting with **Transfer Learning** using a pre-trained Convolutional Neural Network (CNN).

**Why Transfer Learning?**

1.  **Less Data Needed**: Training a deep CNN from scratch requires a very large dataset. With transfer learning, you can achieve good results even with a relatively smaller dataset, as the pre-trained model has already learned powerful features from a massive dataset (like ImageNet).
2.  **Faster Training**: You don't need to train the entire model from scratch, significantly reducing training time and computational resources.
3.  **Good Performance**: Pre-trained models often serve as excellent feature extractors, providing a strong baseline for performance.

**Recommended Pre-trained Models to Start With:**

*   **MobileNetV2**: This is a lightweight and efficient model, great for beginners as it's faster to train and uses less memory. It's a good balance of accuracy and computational cost.
*   **ResNet50**: A more powerful model than MobileNetV2, offering higher accuracy but requiring more computational resources. It's a classic choice for many image classification tasks.
*   **VGG16/VGG19**: These are older but still effective models. They are conceptually simpler to understand but are generally larger and slower than ResNet or MobileNet.

**How to use them (High-Level Steps):**

1.  **Load the pre-trained model**: You'll usually load its convolutional base (without the top classification layers).
2.  **Freeze the base layers**: Prevent the pre-trained layers from being updated during training. This keeps the learned features intact.
3.  **Add your own classification head**: Attach a few new layers (e.g., `Flatten`, `Dense` layers with an appropriate activation function like `softmax` for multi-class classification) on top of the frozen base.
4.  **Train the new layers**: Only train the newly added classification head on your specific dataset.
5.  **(Optional) Fine-tuning**: Unfreeze some of the top layers of the pre-trained base and train them along with your classification head using a very small learning rate. This allows the model to adapt its features slightly to your specific task.

I recommend starting with **MobileNetV2** as it's a great entry point for transfer learning. Once you're comfortable with that, you can experiment with ResNet50.

# Task
Implement and train a ResNet50 model for classifying Bangladeshi bank notes from the image dataset located at `/content/drive/MyDrive/Datasets/ML Datasets/dataset`. The task involves preprocessing the datasets, loading a pre-trained ResNet50 base model with frozen layers, building a new classification head, combining them, compiling the model with appropriate settings, training it, and finally evaluating its performance on the test set.

## Prepare Datasets for Training

### Subtask:
Apply image normalization (rescaling pixel values to [0, 1]) to the datasets and configure them for optimal performance using caching and prefetching.


## Prepare Datasets for Training

### Subtask:
Apply image normalization (rescaling pixel values to [0, 1]) to the datasets and configure them for optimal performance using caching and prefetching.

**Reasoning**:
The next step is to implement the instructions by defining the rescaling layer, applying it to the datasets, caching them, and then applying prefetching for optimal performance.



In [14]:
import tensorflow as tf

# 1. Define a tf.keras.layers.Rescaling layer to normalize pixel values from [0, 255] to [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1./255)

# 2. Apply this normalization layer to the train_ds, val_ds, and test_ds datasets using the .map() method.
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# 3. Cache each of the normalized datasets using the .cache() method.
train_ds = train_ds.cache()
val_ds = val_ds.cache()
test_ds = test_ds.cache()

# 4. Apply prefetching to each cached dataset using the .prefetch() method with tf.data.AUTOTUNE.
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

print("Datasets normalized, cached, and prefetched successfully!")

Datasets normalized, cached, and prefetched successfully!


## Load Pre-trained ResNet50 Base Model

### Subtask:
Load the ResNet50 model from `tf.keras.applications` with `weights='imagenet'` and `include_top=False`, which provides the convolutional base without the final classification layers. This base will serve as a feature extractor.


**Reasoning**:
The subtask requires loading the ResNet50 model, initializing it with specific parameters, and freezing its layers. This code block will perform all these actions.



In [15]:
from tensorflow.keras.applications import ResNet50

# 1. Load the ResNet50 model without the top classification layer
resnet_base = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# 2. Freeze the layers of the base model
resnet_base.trainable = False

print("ResNet50 base model loaded and layers frozen successfully!")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
ResNet50 base model loaded and layers frozen successfully!


## Build the Classification Head

### Subtask:
Create a new classification head consisting of layers like GlobalAveragePooling2D and Dense layers, with the final Dense layer having num_classes outputs and a softmax activation function for multi-class classification.


**Reasoning**:
The subtask is to build a classification head for the ResNet50 model. This involves defining a sequential model with GlobalAveragePooling2D, a Dense layer with ReLU activation, and a final Dense layer with softmax activation based on the number of classes.



In [18]:
import tensorflow as tf

# 1. Get the number of classes
num_classes = len(class_names)

# 2. Create a tf.keras.Sequential model for the classification head
classification_head = tf.keras.Sequential([
    # 3. Add a GlobalAveragePooling2D() layer
    tf.keras.layers.GlobalAveragePooling2D(),
    # 4. Add a Dense layer with ReLU activation
    tf.keras.layers.Dense(128, activation='relu'),
    # 5. Add a final Dense layer with num_classes units and softmax activation
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

print(f"Classification head created with {num_classes} output classes.")
classification_head.summary()

Classification head created with 4 output classes.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Combine Base Model and Classification Head

### Subtask:
Combine the frozen ResNet50 base model (`resnet_base`) with the custom classification head (`classification_head`) using `tf.keras.Sequential` to create the final model for training.

**Reasoning**:
Combine the frozen ResNet50 base model with the custom classification head to create the final model for training. This involves stacking them sequentially.



In [19]:
model = tf.keras.Sequential([
    resnet_base,
    classification_head
])

print("Combined model created successfully!")
model.summary()

Combined model created successfully!


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 4)              │       262,788 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,500 (90.98 MB)

 Trainable params: 262,788 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

## Compile the Model

### Subtask:
Compile the combined `model` using an appropriate optimizer, loss function, and metrics. For multi-class classification, `Adam` optimizer, `SparseCategoricalCrossentropy` loss (since labels are integer-encoded), and `accuracy` metric are common choices.

#### Instructions:
1.  **Select Optimizer**: Use `tf.keras.optimizers.Adam`.
2.  **Select Loss Function**: Use `tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)` because the model's final layer uses `softmax` activation, which outputs probabilities (not logits).
3.  **Select Metrics**: Monitor `accuracy` during training.

**Reasoning**:
The subtask requires compiling the combined model with the specified optimizer, loss function, and metrics.



In [20]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

print("Model compiled successfully!")

Model compiled successfully!


## Train the Model

### Subtask:
Train the compiled `model` using the `train_ds` for training and `val_ds` for validation. Monitor the `accuracy` and `loss` during training.

#### Instructions:
1.  **Set Epochs**: Choose an appropriate number of training epochs (e.g., 10-20 is a good starting point for transfer learning).
2.  **Train the model**: Use `model.fit()` with `train_ds` and `val_ds`.
3.  **Store History**: Store the training history in a variable (e.g., `history`) to visualize training progress later.

**Reasoning**:
Now that the model is compiled, the next logical step is to train it using the prepared `train_ds` and `val_ds` datasets. This involves specifying the number of epochs and capturing the training history.



In [ ]:
EPOCHS = 10 # You can adjust this number

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds
)

print("Model training complete. History stored.")

Epoch 1/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2135s 7s/step - accuracy: 0.3026 - loss: 1.3912 - val_accuracy: 0.4063 - val_loss: 1.2781
Epoch 2/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 1995s 6s/step - accuracy: 0.4037 - loss: 1.2786 - val_accuracy: 0.4540 - val_loss: 1.2241
Epoch 3/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2004s 6s/step - accuracy: 0.4340 - loss: 1.2364 - val_accuracy: 0.4729 - val_loss: 1.1918
Epoch 4/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 1994s 6s/step - accuracy: 0.4547 - loss: 1.2073 - val_accuracy: 0.4826 - val_loss: 1.1616
Epoch 5/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 1988s 6s/step - accuracy: 0.4734 - loss: 1.1803 - val_accuracy: 0.4998 - val_loss: 1.1336
Epoch 6/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2036s 6s/step - accuracy: 0.4910 - loss: 1.1509 - val_accuracy: 0.5095 - val_loss: 1.1117
Epoch 7/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2044s 6s/step - accuracy: 0.5062 - loss: 1.1275 - val_accuracy: 0.5211 - val_loss: 1.0929
Epoch 8/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2011s 6s/step - accuracy: 0.5163 - loss: 1.1044 - 

In [ ]:
model.save('/content/drive/MyDrive/resnet50_trained_model.keras')

NameError: name 'model' is not defined

In [3]:
import tensorflow as tf

# Define the path to the saved model
model_path = '/content/drive/MyDrive/resnet50_trained_model.keras'

# Load the model
loaded_model = tf.keras.models.load_model(model_path)

print(f"Model loaded successfully from {model_path}!")
loaded_model.summary()

Model loaded successfully from /content/drive/MyDrive/resnet50_trained_model.keras!


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 4)              │       262,788 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,376,078 (92.99 MB)

 Trainable params: 262,788 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 525,578 (2.00 MB)

The model has been loaded and its summary is printed above. You can now use `loaded_model` for further evaluation or predictions.

## Evaluate the Model on the Test Set

### Subtask:
Evaluate the `loaded_model` on the `test_ds` to assess its performance on unseen data. This will provide metrics such as loss and accuracy.

#### Instructions:
1.  Use the `loaded_model.evaluate()` method with the `test_ds`.
2.  Print the evaluation results, specifically the test loss and test accuracy.

:**Reasoning**:
Evaluating the model on the test set is crucial to understand its generalization capability. This code will perform the evaluation and print the results.

In [7]:
test_loss, test_accuracy = loaded_model.evaluate(test_ds)

print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

68/68 ━━━━━━━━━━━━━━━━━━━━ 709s 10s/step - accuracy: 0.1202 - loss: 101.7305

Test Loss: 68.5307
Test Accuracy: 0.2423


# Task
Prepare a sample image from `test_ds` for prediction, ensuring it is correctly shaped and normalized.

## Prepare a Sample Image for Prediction

### Subtask:
Select a single image from the `test_ds` dataset and prepare it for prediction. This involves extracting the image, ensuring it has the correct shape (adding a batch dimension), and confirming it's normalized, similar to how training data was processed.


**Reasoning**:
The subtask requires extracting a single image from the `test_ds` and preparing it for prediction by adding a batch dimension. This code block will iterate through the test dataset, take one batch, extract the first image and its label, and then expand the image's dimensions.



In [16]:
import tensorflow as tf

# Get a single batch from the test dataset
for image_batch, label_batch in test_ds.take(1):
    # Extract the first image and its label
    sample_image = image_batch[0]
    sample_label = label_batch[0]

# Add a batch dimension to the sample image
sample_image_expanded = tf.expand_dims(sample_image, 0)

print(f"Original image shape: {sample_image.shape}")
print(f"Expanded image shape (ready for prediction): {sample_image_expanded.shape}")
print(f"Sample label (raw integer): {sample_label.numpy()}")
print(f"Sample label (class name): {class_names[sample_label.numpy()]}")

Original image shape: (224, 224, 3)
Expanded image shape (ready for prediction): (1, 224, 224, 3)
Sample label (raw integer): 0
Sample label (class name): 100


# Task
Interpret the model's probability prediction for the sample image. Identify the class with the highest probability and map it to its human-readable class name using the `class_names` list. Then, present the predicted class label, its confidence score, and compare it with the actual label of the sample image.

## Interpret the Prediction

### Subtask:
Convert the model's probability output into a human-readable class label. This involves finding the class with the highest probability and mapping it back to the original class name using `class_names`.


**Reasoning**:
The subtask requires converting the model's raw probability output into a human-readable class label by finding the class with the highest probability and then mapping that index to its corresponding class name. This code will perform the prediction using the loaded model, then extract and print the predicted class and its confidence.



In [17]:
import numpy as np

# Make a prediction using the loaded model on the expanded sample image
predictions = loaded_model.predict(sample_image_expanded)

# 1. Find the index of the class with the highest probability
predicted_class_index = np.argmax(predictions[0])

# 2. Extract the confidence score (the maximum probability)
confidence_score = predictions[0][predicted_class_index]

# 3. Use the class_names list to get the human-readable predicted class label
predicted_class_name = class_names[predicted_class_index]

# 4. Print the predicted class label and its corresponding confidence score
print(f"\nModel Predictions (probabilities): {predictions[0]}")
print(f"Predicted Class Index: {predicted_class_index}")
print(f"Predicted Class: {predicted_class_name} (Confidence: {confidence_score:.4f})")
print(f"Actual Class: {class_names[sample_label.numpy()]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step

Model Predictions (probabilities): [0.020114   0.02140551 0.01921411 0.9392664 ]
Predicted Class Index: 3
Predicted Class: 500 (Confidence: 0.9393)
Actual Class: 100


## Final Task

### Subtask:
Present the predicted class label for the sample image, along with the actual label if available, and any relevant confidence scores.


## Summary:

### Q&A
*   **Predicted Class Label:** The predicted class label is "500".
*   **Confidence Score:** The confidence score for the predicted class "500" is 0.9393.
*   **Actual Label:** The actual class label for the sample image is "100".

### Data Analysis Key Findings
*   The model predicted the class "500" with a high confidence score of 0.9393.
*   The probabilities for all classes were: \[0.020114, 0.02140551, 0.01921411, 0.9392664]. The highest probability corresponds to the index 3.
*   The actual class of the sample image was "100", which differs from the model's prediction.

### Insights or Next Steps
*   The model confidently misclassified the sample image, predicting "500" instead of the actual "100". This suggests a potential area for further model evaluation, especially on specific misclassified examples.
*   Investigate the features of the misclassified image (actual class "100") and compare them to features typically associated with class "500" to understand the source of the model's error.


# Task
Load MobileNetV2 base model from `tf.keras.applications` with `weights='imagenet'` and `include_top=False`, freezing its layers. Then, combine this frozen base with the previously defined classification head using `tf.keras.Sequential`. Compile the new model with `tf.keras.optimizers.Adam`, `tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)`, and `accuracy` as a metric. Train the compiled model for 5-10 epochs using `train_ds` and `val_ds`, storing the training history. Finally, summarize the training process, including time taken and MobileNetV2 model performance.

## Load MobileNetV2 Base Model

### Subtask:
Load the MobileNetV2 model from `tf.keras.applications` with `weights='imagenet'` and `include_top=False`, providing the convolutional base without the final classification layers. Freeze its layers to use it as a feature extractor. This replaces the previous ResNet50 base model.


**Reasoning**:
The subtask requires loading the MobileNetV2 model, initializing it with specific parameters, and freezing its layers. This code block will perform all these actions.



In [21]:
from tensorflow.keras.applications import MobileNetV2

# 1. Load the MobileNetV2 model without the top classification layer
mobilenet_base = MobileNetV2(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    include_top=False,
    weights='imagenet'
)

# 2. Freeze the layers of the base model
mobilenet_base.trainable = False

print("MobileNetV2 base model loaded and layers frozen successfully!")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
MobileNetV2 base model loaded and layers frozen successfully!


**Reasoning**:
The subtask requires combining the newly loaded MobileNetV2 base model with the existing classification head to form the complete model architecture. This will be done using `tf.keras.Sequential`.



In [22]:
model_mobilenet = tf.keras.Sequential([
    mobilenet_base,
    classification_head
])

print("Combined MobileNetV2 model created successfully!")
model_mobilenet.summary()

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "functional_5" is incompatible with the layer: expected shape=(None, 7, 7, 2048), found shape=(None, 7, 7, 1280)[0m

Arguments received by Sequential.call():
  • args=('<KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, ragged=False, name=keras_tensor_710>',)
  • kwargs={'mask': 'None'}

**Reasoning**:
The previous error occurred because the `classification_head` object was already 'built' with an input shape compatible with ResNet50's output (2048 features). When attempting to combine it with MobileNetV2's output (1280 features), a shape mismatch occurred. To fix this, I will create a new instance of the classification head, ensuring it is built correctly to match MobileNetV2's output shape.



In [23]:
import tensorflow as tf

# Re-create the classification head to ensure it's not pre-built with ResNet50's output shape
# This ensures it adapts to MobileNetV2's output
new_classification_head_for_mobilenet = tf.keras.Sequential([
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model_mobilenet = tf.keras.Sequential([
    mobilenet_base,
    new_classification_head_for_mobilenet
])

print("Combined MobileNetV2 model created successfully!")
model_mobilenet.summary()

Combined MobileNetV2 model created successfully!


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (None, 4)              │       164,484 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,468 (9.24 MB)

 Trainable params: 164,484 (642.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

**Reasoning**:
The next step is to compile the newly created `model_mobilenet` with the specified optimizer, loss function, and metrics, as per the subtask instructions.



In [24]:
model_mobilenet.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

print("MobileNetV2 model compiled successfully!")

MobileNetV2 model compiled successfully!


**Reasoning**:
Now that the MobileNetV2 model is compiled, the next logical step is to train it using the prepared `train_ds` and `val_ds` datasets. This involves specifying the number of epochs (5-10 as per the task) and capturing the training history.



In [25]:
EPOCHS_MOBILENET = 10 # Adjust this number as needed, typically 5-10 for transfer learning

history_mobilenet = model_mobilenet.fit(
    train_ds,
    epochs=EPOCHS_MOBILENET,
    validation_data=val_ds
)

print("MobileNetV2 model training complete. History stored.")

Epoch 1/10
  9/316 ━━━━━━━━━━━━━━━━━━━━ 38:28 8s/step - accuracy: 0.4730 - loss: 1.3559

KeyboardInterrupt: 

# Task
Reduce the size of `train_ds`, `val_ds`, and `test_ds` by applying a `take()` operation to each, and then verify the new number of batches and approximate number of samples for each dataset. Re-compile `model_mobilenet` with `tf.keras.optimizers.Adam`, `tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)`, and `accuracy` metric. Finally, train the re-compiled `model_mobilenet` on the reduced `train_ds` and `val_ds` for 5-10 epochs, storing the training history, and then evaluate its performance on the reduced `test_ds`.

## Reduce Dataset Size

### Subtask:
Modify the existing `train_ds`, `val_ds`, and `test_ds` by applying a `take()` operation to significantly reduce the number of samples in each dataset. This will make training much faster.


**Reasoning**:
The subtask requires modifying the existing datasets by applying a `take()` operation to reduce their size, followed by re-caching and prefetching for optimal performance. This code block will perform all these actions.



In [26]:
import tensorflow as tf

# 1. Define a variable to specify how many batches to take from each dataset
reduced_batches_count = 20 # You can adjust this number

# 2. Apply the .take() method to reduce the number of samples in each dataset
train_ds = train_ds.take(reduced_batches_count)
val_ds = val_ds.take(reduced_batches_count)
test_ds = test_ds.take(reduced_batches_count)

print(f"Datasets reduced to {reduced_batches_count} batches each.")

# 3. Re-apply caching and prefetching to the new, smaller datasets
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

print("Reduced datasets re-cached and prefetched successfully!")

Datasets reduced to 20 batches each.
Reduced datasets re-cached and prefetched successfully!


**Reasoning**:
Now that the datasets have been reduced, it's important to verify their new sizes by checking the number of batches and approximate number of samples, which is part of the overall task instructions.



In [27]:
import tensorflow as tf

# Verify the new number of batches in each dataset
print(f"Number of batches in reduced train_ds: {tf.data.experimental.cardinality(train_ds).numpy()}")
print(f"Number of batches in reduced val_ds: {tf.data.experimental.cardinality(val_ds).numpy()}")
print(f"Number of batches in reduced test_ds: {tf.data.experimental.cardinality(test_ds).numpy()}")

# Calculate and print the approximate number of samples in each reduced dataset
# Note: tf.data.experimental.cardinality() returns the number of batches, not samples
# Multiply by BATCH_SIZE to get an approximate number of samples

# For train_ds, if cardinality is -2, it means the size is unknown, so we use the reduced_batches_count * BATCH_SIZE
num_train_samples_approx = tf.data.experimental.cardinality(train_ds).numpy() * BATCH_SIZE if tf.data.experimental.cardinality(train_ds).numpy() != tf.data.INFINITE_CARDINALITY else reduced_batches_count * BATCH_SIZE
num_val_samples_approx = tf.data.experimental.cardinality(val_ds).numpy() * BATCH_SIZE if tf.data.experimental.cardinality(val_ds).numpy() != tf.data.INFINITE_CARDINALITY else reduced_batches_count * BATCH_SIZE
num_test_samples_approx = tf.data.experimental.cardinality(test_ds).numpy() * BATCH_SIZE if tf.data.experimental.cardinality(test_ds).numpy() != tf.data.INFINITE_CARDINALITY else reduced_batches_count * BATCH_SIZE

print(f"Approximate number of training samples (reduced): {num_train_samples_approx}")
print(f"Approximate number of validation samples (reduced): {num_val_samples_approx}")
print(f"Approximate number of test samples (reduced): {num_test_samples_approx}")

Number of batches in reduced train_ds: 20
Number of batches in reduced val_ds: 20
Number of batches in reduced test_ds: 20
Approximate number of training samples (reduced): 640
Approximate number of validation samples (reduced): 640
Approximate number of test samples (reduced): 640


**Reasoning**:
The next step is to re-compile the `model_mobilenet` as instructed, using the specified optimizer, loss function, and metrics. This ensures the model is ready for training on the reduced datasets.



In [28]:
model_mobilenet.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

print("MobileNetV2 model re-compiled successfully!")

MobileNetV2 model re-compiled successfully!


**Reasoning**:
Now that the MobileNetV2 model is re-compiled and the datasets are reduced, the next logical step is to train it using the prepared `train_ds` and `val_ds` datasets. This involves specifying the number of epochs (5-10 as per the task) and capturing the training history.



In [29]:
EPOCHS_MOBILENET_REDUCED = 10 # Adjust this number as needed, typically 5-10 for transfer learning on reduced dataset

history_mobilenet_reduced = model_mobilenet.fit(
    train_ds,
    epochs=EPOCHS_MOBILENET_REDUCED,
    validation_data=val_ds
)

print("MobileNetV2 model training on reduced datasets complete. History stored.")

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 303s 15s/step - accuracy: 0.7118 - loss: 0.7404 - val_accuracy: 0.8094 - val_loss: 0.4948
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 68s 4s/step - accuracy: 0.8957 - loss: 0.3301 - val_accuracy: 0.8734 - val_loss: 0.3631
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.9700 - loss: 0.1499 - val_accuracy: 0.9078 - val_loss: 0.2761
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.9834 - loss: 0.1033 - val_accuracy: 0.8906 - val_loss: 0.3038
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - accuracy: 0.9967 - loss: 0.0758 - val_accuracy: 0.8938 - val_loss: 0.2730
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.9994 - loss: 0.0548 - val_accuracy: 0.9266 - val_loss: 0.2177
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.9994 - loss: 0.0489 - val_accuracy: 0.9453 - val_loss: 0.1516
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - accuracy: 1.0000 - loss: 0.0398 - val_accuracy: 0.9500 - val_los

**Reasoning**:
Now that the MobileNetV2 model has been trained on the reduced dataset, the next logical step is to evaluate its performance on the `test_ds`, as specified in the task instructions. This will provide insights into its generalization capabilities.



In [30]:
loss_mobilenet, accuracy_mobilenet = model_mobilenet.evaluate(test_ds)

print(f"\nMobileNetV2 Test Loss (reduced dataset): {loss_mobilenet:.4f}")
print(f"MobileNetV2 Test Accuracy (reduced dataset): {accuracy_mobilenet:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.9564 - loss: 0.1282

MobileNetV2 Test Loss (reduced dataset): 0.1867
MobileNetV2 Test Accuracy (reduced dataset): 0.9281


In [34]:
model.save('/content/drive/MyDrive/mobilenetv2_reduced_trained_model.keras')

## MobileNetV2 Training and Evaluation Summary (Reduced Dataset)

### Training Process:
*   **Model**: MobileNetV2 with a custom classification head.
*   **Epochs**: 10 (as defined by `EPOCHS_MOBILENET_REDUCED`).
*   **Training Dataset**: `train_ds` reduced to 20 batches (approx. 640 samples).
*   **Validation Dataset**: `val_ds` reduced to 20 batches (approx. 640 samples).
*   **Optimizer**: Adam.
*   **Loss Function**: SparseCategoricalCrossentropy.
*   **Metrics**: Accuracy.

### Training Performance Highlights (from `history_mobilenet_reduced`):
*   **Epoch 1**: Train Accuracy: 0.7118, Train Loss: 0.7404 | Val Accuracy: 0.8094, Val Loss: 0.4948
*   **Epoch 10**: Train Accuracy: 0.9973, Train Loss: 0.0297 | Val Accuracy: 0.9422, Val Loss: 0.1476

### Overall Training Insight:
The model trained very quickly on the reduced dataset (each epoch took roughly 1 minute on average, compared to several minutes per epoch for the full dataset). The training accuracy rapidly approached 1.0, and validation accuracy also showed significant improvement, reaching over 94% by the final epoch. This indicates that the MobileNetV2 base, even with a small custom head, is highly effective for transfer learning on this dataset, even with limited samples.

### Evaluation on Reduced Test Set:
*   **Test Loss**: 0.1867
*   **Test Accuracy**: 0.9281

### Conclusion:
The MobileNetV2 model demonstrated strong performance on the reduced test dataset, achieving an accuracy of approximately 92.81%. This confirms its suitability for the task, especially when computational resources or training time are limited. The rapid convergence and high accuracy on the validation and test sets highlight the efficiency of transfer learning with MobileNetV2.

## Final Task

### Subtask:
Summarize the results of training with a reduced dataset, comparing training time and performance to previous attempts, and discuss implications for model development.


## Summary:

### Q&A
*   **Comparison of training time and performance with a reduced dataset:** Training on the reduced dataset was significantly faster, with each epoch taking approximately 1 minute, compared to several minutes per epoch for the full dataset (though explicit full dataset performance was not provided in this specific analysis). The model achieved a high validation accuracy of 94.22% by the 10th epoch and a test accuracy of 92.81% on the reduced dataset.
*   **Implications for model development:** The rapid convergence and strong performance (92.81% test accuracy) on the reduced dataset highlight the efficiency and effectiveness of transfer learning with MobileNetV2. This approach is highly suitable for scenarios where computational resources are limited or faster prototyping and development cycles are desired, as it still yields a high-performing model with fewer samples.

### Data Analysis Key Findings
*   The `train_ds`, `val_ds`, and `test_ds` datasets were successfully reduced to 20 batches each using the `.take()` operation. This resulted in approximately 640 samples per dataset (assuming a `BATCH_SIZE` of 32).
*   The MobileNetV2 model was re-compiled with `tf.keras.optimizers.Adam()` and `tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)`, using `accuracy` as the metric.
*   After 10 epochs of training on the reduced datasets:
    *   The training accuracy improved from 0.7118 (Epoch 1) to 0.9973 (Epoch 10).
    *   The validation accuracy improved from 0.8094 (Epoch 1) to 0.9422 (Epoch 10).
    *   Each epoch completed in approximately 1 minute.
*   Evaluation on the reduced test dataset yielded a test loss of 0.1867 and a test accuracy of 0.9281.

### Insights or Next Steps
*   Transfer learning with MobileNetV2 is highly effective for this classification task, demonstrating strong performance (92.81% test accuracy) even when trained on a significantly reduced dataset.
*   The ability to achieve high accuracy with fewer samples and faster training times makes this approach valuable for rapid prototyping and deployment in resource-constrained environments.


# Task
Reload the ResNet50 model from `tf.keras.applications` with `weights='imagenet'` and `include_top=False`, ensuring its layers are frozen.

## Reload ResNet50 Base Model

### Subtask:
Load the ResNet50 model from `tf.keras.applications` with `weights='imagenet'` and `include_top=False`, ensuring its layers are frozen. This will be a fresh instance to avoid conflicts with previous model definitions.


**Reasoning**:
The subtask requires reloading the ResNet50 model with frozen layers to ensure a fresh instance for further use. This involves importing the necessary class, initializing the model, and then freezing its weights.



In [31]:
from tensorflow.keras.applications import ResNet50

# 1. Load the ResNet50 model without the top classification layer
resnet_base_reloaded = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# 2. Freeze the layers of the reloaded base model
resnet_base_reloaded.trainable = False

print("ResNet50 base model reloaded and layers frozen successfully!")

ResNet50 base model reloaded and layers frozen successfully!


## Build a New Classification Head for ResNet50

### Subtask:
Create a new classification head for the ResNet50 model, consisting of GlobalAveragePooling2D and Dense layers with appropriate activation, ensuring it matches the expected output of the ResNet50 base.


**Reasoning**:
The subtask requires creating a new classification head for the ResNet50 model, including GlobalAveragePooling2D and Dense layers with appropriate activation. This code block will define and summarize the new classification head.



In [32]:
import tensorflow as tf

# Get the number of classes (already defined as num_classes)
# num_classes = len(class_names)

# Create a new tf.keras.Sequential model for the ResNet50 classification head
resnet_classification_head = tf.keras.Sequential([
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

print(f"New classification head for ResNet50 created with {num_classes} output classes.")
resnet_classification_head.summary()

New classification head for ResNet50 created with 4 output classes.


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ global_average_pooling2d_2      │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Combine ResNet50 Base and New Classification Head

### Subtask:
Combine the reloaded frozen ResNet50 base model with the newly created classification head using tf.keras.Sequential to form the complete ResNet50 model for training.


**Reasoning**:
Combine the reloaded frozen ResNet50 base model with the new classification head to create the complete ResNet50 model for training, then print its summary.



In [33]:
model_resnet = tf.keras.Sequential([
    resnet_base_reloaded,
    resnet_classification_head
])

print("Combined ResNet50 model created successfully!")
model_resnet.summary()

Combined ResNet50 model created successfully!


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_5 (Sequential)       │ (None, 4)              │       262,788 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,500 (90.98 MB)

 Trainable params: 262,788 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

## Compile the ResNet50 Model

### Subtask:
Compile the combined `model_resnet` using an appropriate optimizer, loss function, and metrics. For multi-class classification, `Adam` optimizer, `SparseCategoricalCrossentropy` loss (since labels are integer-encoded), and `accuracy` metric are common choices.

#### Instructions:
1.  **Select Optimizer**: Use `tf.keras.optimizers.Adam()`.
2.  **Select Loss Function**: Use `tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)` because the model's final layer uses `softmax` activation, which outputs probabilities (not logits).
3.  **Select Metrics**: Monitor `accuracy` during training.

**Reasoning**:
The previous code block successfully combined the ResNet50 base model with the new classification head. The next logical step is to compile this newly formed model, specifying the optimizer, loss function, and metrics, as part of the overall task workflow.



In [35]:
model_resnet.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

print("ResNet50 model compiled successfully!")

ResNet50 model compiled successfully!


## Train the ResNet50 Model

### Subtask:
Train the compiled `model_resnet` using the `train_ds` for training and `val_ds` for validation. Monitor the `accuracy` and `loss` during training.

#### Instructions:
1.  **Set Epochs**: Choose an appropriate number of training epochs (e.g., 5-10 is a good starting point for transfer learning on a reduced dataset).
2.  **Train the model**: Use `model_resnet.fit()` with the reduced `train_ds` and `val_ds`.
3.  **Store History**: Store the training history in a variable (e.g., `history_resnet`) to compare with previous training runs.

**Reasoning**:
Now that the ResNet50 model is compiled, the next logical step is to train it using the prepared (reduced) `train_ds` and `val_ds` datasets, as per the task instructions.



In [36]:
EPOCHS_RESNET = 10 # You can adjust this number, typically 5-10 for reduced datasets

history_resnet = model_resnet.fit(
    train_ds,
    epochs=EPOCHS_RESNET,
    validation_data=val_ds
)

print("ResNet50 model training complete. History stored.")

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 244s 12s/step - accuracy: 0.2791 - loss: 1.4988 - val_accuracy: 0.1109 - val_loss: 1.4103
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 256s 12s/step - accuracy: 0.2696 - loss: 1.4031 - val_accuracy: 0.7828 - val_loss: 1.1002
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 233s 12s/step - accuracy: 0.2978 - loss: 1.3690 - val_accuracy: 0.7141 - val_loss: 1.1480
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 287s 13s/step - accuracy: 0.3408 - loss: 1.3472 - val_accuracy: 0.6859 - val_loss: 1.1678
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 261s 13s/step - accuracy: 0.3432 - loss: 1.3292 - val_accuracy: 0.6391 - val_loss: 1.1926
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 230s 12s/step - accuracy: 0.3656 - loss: 1.3148 - val_accuracy: 0.6219 - val_loss: 1.1950
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 231s 12s/step - accuracy: 0.3689 - loss: 1.3007 - val_accuracy: 0.6062 - val_loss: 1.1899
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 230s 12s/step - accuracy: 0.3721 - loss: 1.2888 - val_accuracy: 0.

In [41]:
model.save('/content/drive/MyDrive/resnet50_reduced_trained_model.keras')

## Evaluate the ResNet50 Model on the Test Set

### Subtask:
Evaluate the `model_resnet` on the reduced `test_ds` to assess its performance on unseen data. This will provide metrics such as loss and accuracy.

#### Instructions:
1.  Use the `model_resnet.evaluate()` method with the `test_ds`.
2.  Print the evaluation results, specifically the test loss and test accuracy.

**Reasoning**:
Now that the ResNet50 model has been trained, the next step is to evaluate its performance on the test set using the `evaluate` method and then print the loss and accuracy.



In [37]:
loss_resnet, accuracy_resnet = model_resnet.evaluate(test_ds)

print(f"\nResNet50 Test Loss (reduced dataset): {loss_resnet:.4f}")
print(f"ResNet50 Test Accuracy (reduced dataset): {accuracy_resnet:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 115s 6s/step - accuracy: 0.5985 - loss: 1.1744

ResNet50 Test Loss (reduced dataset): 1.1941
ResNet50 Test Accuracy (reduced dataset): 0.5719


## ResNet50 Training and Evaluation Summary (Reduced Dataset)

### Training Process:
*   **Model**: ResNet50 with a custom classification head.
*   **Epochs**: 10 (as defined by `EPOCHS_RESNET`).
*   **Training Dataset**: `train_ds` reduced to 20 batches (approx. 640 samples).
*   **Validation Dataset**: `val_ds` reduced to 20 batches (approx. 640 samples).
*   **Optimizer**: Adam.
*   **Loss Function**: SparseCategoricalCrossentropy.
*   **Metrics**: Accuracy.

### Training Performance Highlights (from `history_resnet`):
*   **Epoch 1**: Train Accuracy: 0.2791, Train Loss: 1.4988 | Val Accuracy: 0.1109, Val Loss: 1.4103
*   **Epoch 10**: Train Accuracy: 0.4147, Train Loss: 1.2653 | Val Accuracy: 0.5375, Val Loss: 1.1844

### Overall Training Insight:
Training the ResNet50 model on the reduced dataset took significantly longer per epoch (around 12-13 seconds per step for ~20 steps, totaling around 4-5 minutes per epoch) compared to MobileNetV2 on the reduced dataset (around 3-4 seconds per step). The training and validation accuracy showed improvement over epochs, but the final validation accuracy was 53.75%, which is considerably lower than MobileNetV2's 94.22% on the same reduced dataset. This suggests that while ResNet50 is a powerful model, its performance on this specific reduced dataset with a simple classification head is not as effective as MobileNetV2, perhaps due to its larger parameter count making it harder to fine-tune with very limited data or requiring more elaborate fine-tuning strategies.

### Evaluation on Reduced Test Set:
*   **Test Loss**: 1.1941
*   **Test Accuracy**: 0.5719

### Conclusion:
The ResNet50 model performed relatively poorly on the reduced test dataset, achieving an accuracy of approximately 57.19%. This is a notable difference compared to MobileNetV2's performance (92.81%) on the same reduced dataset. The choice of base model and dataset size significantly impacts performance and training efficiency, and MobileNetV2 appears to be a more suitable choice for this task under resource-constrained/reduced data conditions due to its efficiency and better performance.

## Summary: Comparing MobileNetV2 and ResNet50 on Reduced Dataset

### Q&A
*   **Comparison of training time and performance with a reduced dataset (MobileNetV2 vs. ResNet50):**
    *   **MobileNetV2 (Reduced Dataset):** Training was very fast, with each epoch completing in approximately 1 minute. It achieved a high validation accuracy of 94.22% by the 10th epoch and a test accuracy of 92.81%.
    *   **ResNet50 (Reduced Dataset):** Training was significantly slower per epoch, taking around 4-5 minutes per epoch. The model's validation accuracy reached only 53.75% by the 10th epoch, and its test accuracy was 57.19%.
    *   **Conclusion:** MobileNetV2 significantly outperformed ResNet50 in both training speed and accuracy on the reduced dataset.

*   **Implications for model development:** The rapid convergence and strong performance of MobileNetV2 (92.81% test accuracy) on a significantly reduced dataset highlight its efficiency and effectiveness for transfer learning in resource-constrained environments or when rapid prototyping is needed. In contrast, ResNet50's poorer performance on the reduced dataset suggests that while it is a powerful model, its larger parameter count may make it harder to fine-tune effectively with very limited data, potentially requiring more data, more epochs, or a different fine-tuning strategy (e.g., unfreezing more layers) to achieve comparable results. For this specific task and dataset size, MobileNetV2 is clearly the more suitable and efficient choice.

### Data Analysis Key Findings
*   Both MobileNetV2 and ResNet50 were trained on datasets reduced to 20 batches each (approximately 640 samples per dataset).
*   **MobileNetV2 Results:**
    *   Training Accuracy: ~0.9973
    *   Validation Accuracy: ~0.9422
    *   Test Accuracy: 0.9281
    *   Epoch Training Time: ~1 minute
*   **ResNet50 Results:**
    *   Training Accuracy: ~0.4147
    *   Validation Accuracy: ~0.5375
    *   Test Accuracy: 0.5719
    *   Epoch Training Time: ~4-5 minutes

### Insights or Next Steps
*   MobileNetV2 is highly recommended for this image classification task, especially with limited data or computational resources, due to its superior balance of speed and accuracy compared to ResNet50 under these conditions.
*   For ResNet50, further experimentation might include increasing the number of training epochs, implementing learning rate schedules, or exploring fine-tuning by unfreezing some of its top layers, though this would increase training time and require more careful hyperparameter tuning.
*   The significantly different performance between the two models emphasizes the importance of selecting an appropriate pre-trained base model based on the dataset size and available resources.

## Final Task

### Subtask:
Summarize the training process and performance of the ResNet50 model on the reduced dataset, comparing it with the MobileNetV2 results and discussing any notable differences or insights.


## Summary:

### Q&A
*   **Comparison of training time and performance with a reduced dataset (MobileNetV2 vs. ResNet50):**
    *   **MobileNetV2 (Reduced Dataset):** Training was very fast, with each epoch completing in approximately 1 minute. It achieved a high validation accuracy of 94.22% by the 10th epoch and a test accuracy of 92.81%.
    *   **ResNet50 (Reduced Dataset):** Training was significantly slower per epoch, taking around 4-5 minutes per epoch. The model's validation accuracy reached only 53.75% by the 10th epoch, and its test accuracy was 57.19%.
    *   **Conclusion:** MobileNetV2 significantly outperformed ResNet50 in both training speed and accuracy on the reduced dataset.

*   **Implications for model development:** The rapid convergence and strong performance of MobileNetV2 (92.81% test accuracy) on a significantly reduced dataset highlight its efficiency and effectiveness for transfer learning in resource-constrained environments or when rapid prototyping is needed. In contrast, ResNet50's poorer performance on the reduced dataset suggests that while it is a powerful model, its larger parameter count may make it harder to fine-tune effectively with very limited data, potentially requiring more data, more epochs, or a different fine-tuning strategy (e.g., unfreezing more layers) to achieve comparable results. For this specific task and dataset size, MobileNetV2 is clearly the more suitable and efficient choice.

### Data Analysis Key Findings
*   Both MobileNetV2 and ResNet50 were trained on datasets reduced to 20 batches each (approximately 640 samples per dataset).
*   **MobileNetV2 Results:**
    *   Training Accuracy: \~0.9973
    *   Validation Accuracy: \~0.9422
    *   Test Accuracy: 0.9281
    *   Epoch Training Time: \~1 minute
*   **ResNet50 Results:**
    *   Training Accuracy: \~0.4147
    *   Validation Accuracy: \~0.5375
    *   Test Accuracy: 0.5719
    *   Epoch Training Time: \~4-5 minutes

### Insights or Next Steps
*   MobileNetV2 is highly recommended for this image classification task, especially with limited data or computational resources, due to its superior balance of speed and accuracy compared to ResNet50 under these conditions.
*   For ResNet50, further experimentation might include increasing the number of training epochs, implementing learning rate schedules, or exploring fine-tuning by unfreezing some of its top layers, though this would increase training time and require more careful hyperparameter tuning.


# Task
Present the predicted class label, its confidence score, and compare it with the actual label of the sample image for the MobileNetV2 model's prediction.

## Prepare Sample Image

### Subtask:
Ensure a sample image from the test dataset is prepared and formatted correctly for prediction (i.e., with a batch dimension). The `sample_image_expanded` and `sample_label` variables from the previous prediction task can be reused.


**Reasoning**:
Verify that the `sample_image_expanded` and `sample_label` variables are still available in the environment and correctly represent a prepared sample from the `test_ds` by re-printing their shapes and values.



In [38]:
print(f"Verified expanded image shape: {sample_image_expanded.shape}")
print(f"Verified sample label (raw integer): {sample_label.numpy()}")
print(f"Verified sample label (class name): {class_names[sample_label.numpy()]}")

Verified expanded image shape: (1, 224, 224, 3)
Verified sample label (raw integer): 0
Verified sample label (class name): 100


## Make MobileNetV2 Prediction

### Subtask:
Use the `model_mobilenet.predict()` method on the prepared `sample_image_expanded` to obtain the model's probability predictions for each class.


**Reasoning**:
The subtask requires using the `model_mobilenet.predict()` method on the prepared `sample_image_expanded` and storing the results in `mobilenet_predictions`, then printing them.



In [39]:
mobilenet_predictions = model_mobilenet.predict(sample_image_expanded)

print("MobileNetV2 Model Predictions (probabilities for each class):")
print(mobilenet_predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
MobileNetV2 Model Predictions (probabilities for each class):
[[9.9972612e-01 3.4439898e-05 1.9286298e-07 2.3923085e-04]]


**Reasoning**:
The previous step successfully generated the probability predictions. Now, I need to interpret these predictions to find the predicted class, its confidence score, and compare it with the actual label, as per the main task.



In [40]:
import numpy as np

# 1. Find the index of the class with the highest probability
mobilenet_predicted_class_index = np.argmax(mobilenet_predictions[0])

# 2. Extract the confidence score (the maximum probability)
mobilenet_confidence_score = mobilenet_predictions[0][mobilenet_predicted_class_index]

# 3. Use the class_names list to get the human-readable predicted class label
mobilenet_predicted_class_name = class_names[mobilenet_predicted_class_index]

# 4. Print the predicted class label and its corresponding confidence score
print(f"\nMobileNetV2 Predicted Class Index: {mobilenet_predicted_class_index}")
print(f"MobileNetV2 Predicted Class: {mobilenet_predicted_class_name} (Confidence: {mobilenet_confidence_score:.4f})")
print(f"Actual Class: {class_names[sample_label.numpy()]}")


MobileNetV2 Predicted Class Index: 0
MobileNetV2 Predicted Class: 100 (Confidence: 0.9997)
Actual Class: 100


## Final Task

### Subtask:
Present the predicted class label, its confidence score, and compare it with the actual label of the sample image for the MobileNetV2 model's prediction.


## Summary:

### Q&A
The predicted class label for the sample image using the MobileNetV2 model is '100' with a high confidence score of 0.9997. This prediction matches the actual label of the sample image, which is also '100'.

### Data Analysis Key Findings
*   The sample image was successfully prepared and formatted, ensuring it had the correct shape of (1, 224, 224, 3) for model input, including a batch dimension.
*   The actual label for the sample image was identified as '100'.
*   The MobileNetV2 model predicted the class '100' for the sample image.
*   The confidence score associated with this prediction was very high, at 0.9997 (or 99.97%).
*   The model's prediction ('100') perfectly matched the actual label ('100') for the sample image.

### Insights or Next Steps
*   The MobileNetV2 model demonstrated strong performance by accurately classifying the sample image with very high confidence.
*   Further evaluation on a larger test set is recommended to confirm this high accuracy across various samples and classes.
